<font size="6">Football Market Value Prediction using Machine Learning</font>

<font size="4">1. Data Profiling</font>

In [15]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import zipfile
import pandasql as psql

In [16]:
data = {}
for dirname, _, filenames in os.walk('./data/'):
    for filename in filenames:
        if filename[-4:] == '.csv':
            data[str(filename [:-4])] = pd.read_csv(os.path.join(dirname, filename))

Players Dataset

In [17]:
data['players'].head(10)

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,foot,height_in_cm,market_value_in_eur,highest_market_value_in_eur,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name
0,597,Aleksandr,Hleb,Aleksandr Hleb,2016,2696,aleksandr-hleb,UdSSR,Minsk,Belarus,...,NaN,NaN,NaN,19000000.0,NaN,fair-sport GmbH,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aleksandr-hleb...,RU1,Krylya Sovetov Samara
1,1428,Mike,Hanke,Mike Hanke,2013,60,mike-hanke,Germany,Hamm,Germany,...,right,185.0,NaN,5000000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/mike-hanke/pro...,L1,SC Freiburg
2,1560,Youssef,El Akchaoui,Youssef El Akchaoui,2012,306,youssef-el-akchaoui,Netherlands,Dordrecht,Morocco,...,NaN,NaN,NaN,1750000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/youssef-el-akc...,NL1,SC Heerenveen
3,1587,Mario,Eggimann,Mario Eggimann,2012,42,mario-eggimann,Switzerland,Brugg,Switzerland,...,right,190.0,NaN,3000000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/mario-eggimann...,L1,NaN
4,1726,Heiko,Westermann,Heiko Westermann,2016,610,heiko-westermann,Germany,Alzenau,Germany,...,right,190.0,NaN,9000000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/heiko-westerma...,NL1,Ajax Amsterdam
5,2306,Markus,Miller,Markus Miller,2014,42,markus-miller,Germany,Lindenberg im Allgäu,Germany,...,right,189.0,NaN,3500000.0,NaN,ARP Sportmarketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/markus-miller/...,L1,NaN
6,2587,Elyasa,Süme,Elyasa Süme,2016,524,elyasa-sume,Germany,Wesel,Turkey,...,NaN,NaN,NaN,2000000.0,NaN,NG360,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/elyasa-sume/pr...,TR1,NaN
7,2805,Christian,Pander,Christian Pander,2014,42,christian-pander,Germany,Münster,Germany,...,NaN,NaN,NaN,6500000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/christian-pand...,L1,NaN
8,2878,Yassin,Mikari,Yassin Mikari,2012,750,yassin-mikari,Switzerland,Zürich,Tunisia,...,left,172.0,NaN,2800000.0,NaN,Rhein AG,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/yassin-mikari/...,FR1,NaN
9,2899,Hervé,Tum,Hervé Tum,2012,2292,herve-tum,Cameroon,Douala,Cameroon,...,right,184.0,NaN,2600000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/herve-tum/prof...,TR1,NaN


Player Valuations Dataset

In [18]:
data['player_valuations'].head(10)

,player_id,last_season,datetime,date,dateweek,market_value_in_eur,n,current_club_id,player_club_domestic_competition_id
0,3132,2013,2003-12-09 00:00:00,2003-12-09,2003-12-08,400000,1,126,TR1
1,6893,2012,2003-12-15 00:00:00,2003-12-15,2003-12-15,900000,1,984,GB1
2,10,2015,2004-10-04 00:00:00,2004-10-04,2004-10-04,7000000,1,398,IT1
3,26,2017,2004-10-04 00:00:00,2004-10-04,2004-10-04,1500000,1,16,L1
4,65,2015,2004-10-04 00:00:00,2004-10-04,2004-10-04,8000000,1,1091,GR1
5,77,2012,2004-10-04 00:00:00,2004-10-04,2004-10-04,13000000,1,506,IT1
6,80,2017,2004-10-04 00:00:00,2004-10-04,2004-10-04,400000,1,27,L1
7,109,2012,2004-10-04 00:00:00,2004-10-04,2004-10-04,9500000,1,825,TR1
8,123,2012,2004-10-04 00:00:00,2004-10-04,2004-10-04,9500000,1,33,L1
9,132,2015,2004-10-04 00:00:00,2004-10-04,2004-10-04,13000000,1,11,GB1


Clubs Dataset

In [19]:
data['clubs'].head(10)

,club_id,club_code,name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,url
0,127,sc-paderborn-07,NaN,L1,NaN,26,25.7,5,19.2,1,Home Deluxe Arena,15000,€-450k,NaN,2019,https://www.transfermarkt.co.uk/sc-paderborn-0...
1,192,roda-jc-kerkrade,NaN,NL1,NaN,25,23.9,9,36.0,0,Parkstad Limburg Stadion,19979,+€1.30m,NaN,2017,https://www.transfermarkt.co.uk/roda-jc-kerkra...
2,2239,us-cremonese,US Cremonese,IT1,NaN,26,26.0,12,46.2,7,Giovanni Zini,16003,€-33.80m,NaN,2022,https://www.transfermarkt.co.uk/us-cremonese/s...
3,2423,cd-santa-clara,CD Santa Clara,PO1,NaN,31,25.8,20,64.5,2,Estádio de São Miguel,15000,+€5.85m,NaN,2022,https://www.transfermarkt.co.uk/cd-santa-clara...
4,24245,umraniyespor,Ümraniyespor,TR1,NaN,24,28.4,10,41.7,3,Ümraniye Belediyesi Şehir Stadı,3513,+-0,NaN,2022,https://www.transfermarkt.co.uk/umraniyespor/s...
5,2553,kilmarnock-fc,Kilmarnock FC,SC1,NaN,27,25.7,16,59.3,1,"BBSP Stadium, Rugby Park",18128,+-0,NaN,2022,https://www.transfermarkt.co.uk/kilmarnock-fc/...
6,26459,nk-veres-rivne,NaN,UKR1,NaN,22,27.1,0,NaN,0,Stadion Avangard (2022),10000,+€170k,NaN,2022,https://www.transfermarkt.co.uk/nk-veres-rivne...
7,2671,pas-giannina,PAS Giannina,GR1,NaN,31,25.4,14,45.2,0,Stadio Zosimades,7652,+€2.68m,NaN,2022,https://www.transfermarkt.co.uk/pas-giannina/s...
8,2919,ac-monza,AC Monza,IT1,NaN,30,26.8,9,30.0,4,U-Power Stadium - Brianteo,15039,€-46.48m,NaN,2022,https://www.transfermarkt.co.uk/ac-monza/start...
9,3205,kayserispor,Kayserispor,TR1,NaN,28,25.6,13,46.4,3,RHG Enertürk Enerji Stadyumu,32864,+€3.00m,NaN,2022,https://www.transfermarkt.co.uk/kayserispor/st...


Games Dataset

In [20]:
data['games'].head(10)

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,home_club_manager_name,away_club_manager_name,stadium,attendance,referee,url,home_club_name,away_club_name,aggregate,competition_type
0,2222539,L1,2012,1. Matchday,2012-08-25,60,39,1,1,11,...,Christian Streich,Thomas Tuchel,Dreisamstadion,22500.0,Deniz Aytekin,https://www.transfermarkt.co.uk/sc-freiburg_1-...,SC Freiburg,1.FSV Mainz 05,1:1,domestic_league
1,2222541,L1,2012,1. Matchday,2012-08-25,41,4,0,1,15,...,Thorsten Fink,Dieter Hecking,Volksparkstadion,50123.0,Marco Fritz,https://www.transfermarkt.co.uk/hamburger-sv_1...,NaN,NaN,0:1,domestic_league
2,2222542,L1,2012,1. Matchday,2012-08-25,24,15,2,1,5,...,Armin Veh,Sascha Lewandowski,Commerzbank Arena,27950.0,Thorsten Kinhöfer,https://www.transfermarkt.co.uk/eintracht-fran...,Eintracht Frankfurt,Bayer 04 Leverkusen,2:1,domestic_league
3,2222595,RU1,2012,3. Matchday,2012-08-05,2700,4128,1,0,3,...,Guus Hiddink,Rustem Khuzin,Dinamo,14000.0,Vladislav Bezborodov,https://www.transfermarkt.co.uk/anzhi-makhachk...,NaN,NaN,1:0,domestic_league
4,2222636,RU1,2012,4. Matchday,2012-08-12,1083,2696,1,2,15,...,Miodrag Bozovic,Andrey Kobelev,Olimp-2,6800.0,Vitali Meshkov,https://www.transfermarkt.co.uk/fk-rostov_kryl...,FK Rostov,Krylya Sovetov Samara,1:2,domestic_league
5,2222651,RU1,2012,11. Matchday,2012-10-06,12438,11126,0,2,16,...,Gadzhi Gadzhiev,Fedor Shcherbachenko,Zentralstadion Lokomotiv,3100.0,Sergey Karasev,https://www.transfermarkt.co.uk/volga-nizhniy-...,NaN,NaN,0:2,domestic_league
6,2222652,RU1,2012,11. Matchday,2012-10-07,121,2700,0,2,14,...,Dan Petrescu,Guus Hiddink,Arena Khimki,7712.0,Aleksandr Egorov,https://www.transfermarkt.co.uk/dinamo-moscow_...,Dynamo Moscow,NaN,0:2,domestic_league
7,2222675,RU1,2012,12. Matchday,2012-10-21,2410,2698,2,0,2,...,Leonid Slutski,Kurban Berdyev,Arena Khimki,8764.0,Aleksandr Egorov,https://www.transfermarkt.co.uk/cska-moscow_ru...,CSKA Moscow,NaN,2:0,domestic_league
8,2222676,RU1,2012,8. Matchday,2012-09-16,2700,16704,5,2,3,...,Guus Hiddink,Slavoljub Muslin,Dinamo,15000.0,Vitali Meshkov,https://www.transfermarkt.co.uk/anzhi-makhachk...,NaN,FK Krasnodar,5:2,domestic_league
9,2222695,RU1,2012,14. Matchday,2012-11-02,964,1083,2,1,3,...,Luciano Spalletti,Miodrag Bozovic,Petrovsky,20000.0,Maksim Layushkin,https://www.transfermarkt.co.uk/zenit-st-peter...,Zenit St. Petersburg,FK Rostov,2:1,domestic_league


Game Events Dataset

In [23]:
data['game_events'].head(20)

,game_id,minute,type,club_id,player_id,description,player_in_id
0,2221747,70,Substitutions,52,2125,NaN,43984.0
1,2221747,80,Substitutions,440,23566,NaN,40679.0
2,2221747,86,Substitutions,52,8639,NaN,42749.0
3,2221747,88,Substitutions,440,43822,NaN,83191.0
4,2221760,68,Substitutions,15,61087,NaN,39102.0
5,2221760,77,Substitutions,113,19583,NaN,120267.0
6,2221760,85,Substitutions,113,43441,NaN,214304.0
7,2221760,85,Substitutions,15,6237,NaN,89708.0
8,2222539,49,Goals,60,36182,", Left-footed shot, 1. Goal of the Season",NaN
9,2222539,65,Goals,39,2923,", Penalty, 1. Goal of the Season",NaN


Appearances Dataset

In [21]:
data['appearances'].head(10)

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
0,-1_172127,-1,172127,3999,3999,NaN,Tyronne,GRP,0,0,0,0,90
1,-1_210988,-1,210988,3999,553,NaN,Christos Eleftheriadis,GRP,0,0,0,0,90
2,-1_231471,-1,231471,3999,3999,NaN,Theofanis Tzandaris,GRP,0,0,0,0,90
3,-1_256819,-1,256819,3999,3999,NaN,Nikolaos Tsoukalos,GRP,0,0,0,0,90
4,-1_281919,-1,281919,3999,3999,NaN,Adam Tzanetopoulos,GRP,0,0,0,0,90
5,-1_295755,-1,295755,3999,3999,NaN,Georgios Saramantas,GRP,0,0,0,0,90
6,-1_339896,-1,339896,3999,3999,NaN,Kyriakos Mazoulouxis,GRP,0,0,0,0,90
7,-1_36845,-1,36845,3999,3999,NaN,Bojan Saranov,GRP,0,0,0,0,90
8,-1_46728,-1,46728,3999,3999,NaN,Savvas Gentsoglou,GRP,0,0,0,0,90
9,-1_522669,-1,522669,3999,3999,NaN,Georgios Manousakis,GRP,0,0,0,0,90


<font size="4">2. Players Stat Processing</font>

In [29]:
def player_stats(player_name, season):
    
    df = data['appearances'].merge(data['games'], on=['game_id'], how='left')
    df = df[df['player_name'] == player_name]
    df = df[df['season'] == season]
    
    if (df.shape[0] == 0):
        Out = [(np.nan,player_name, season,0,0,0,0,0,0,0)]
        out_df = pd.DataFrame(data = Out, columns = ['player_id','names','season','goals','games',
                                                     'assists','minutes_played','goals_for','goals_against','clean_sheet'])
        return out_df
    
    else:
        
        df["goals_for"] = df.apply(lambda row: row['home_club_goals'] if row['home_club_id'] == row['player_club_id'] 
                      else row['away_club_goals'] if row['away_club_id'] == row['player_club_id'] 
                      else np.nan, axis=1)
        df["goals_against"] = df.apply(lambda row: row['away_club_goals'] if row['home_club_id'] == row['player_club_id'] 
                      else row['home_club_goals'] if row['away_club_id'] == row['player_club_id'] 
                      else np.nan, axis=1)
        df['clean_sheet'] = df.apply(lambda row: 1 if row['goals_against'] == 0
                      else 0 if row['goals_against'] > 0
                      else np.nan, axis=1)
        
        df = df.groupby(['player_id',"name","season"],as_index=False).agg({'goals': 'sum', 'game_id': 'nunique', 
                                                                      'assists': 'sum', 'minutes_played' : 'sum', 'goals_for' : 'sum',
                                                                      'goals_against' : 'sum', 'clean_sheet' : 'sum'})
        out_df = df.rename(columns={'game_id': 'games'})

        return out_df

In [31]:
player_stats('Cristiano Ronaldo', 2019)

,player_id,names,season,goals,games,assists,minutes_played,goals_for,goals_against,clean_sheet
0,NaN,Cristiano Ronaldo,2019,0,0,0,0,0,0,0
